In [3]:
from requests import get
from json import loads
from services.db_service import DbService

In [4]:
db_service = DbService()

In [5]:
def get_card_scryfall_name_and_color(name):
    card = loads(
        get(f"https://api.scryfall.com/cards/named?exact={name}").text)

    type = card["type_line"]
    name = card["name"]
    if "Land" in type:
        return name, "L"

    if "colors" in card:
        colors = card["colors"]
    else:
        colors = card["card_faces"][0]["colors"]

    if len(colors) == 0:
        return name, "C"
    elif len(colors) > 1:
        return name, "M"
    else:
        return name, colors[0]

In [6]:
pool_collection = db_service.pool_collection()


def add_card_or_update(card):
    name = card["name"]
    scryfall_name, color = get_card_scryfall_name_and_color(name)

    existing_card = pool_collection.find_one({"name": name})
    if existing_card and existing_card["count"] == card["count"]:
        return

    if existing_card and existing_card["count"] != card["count"]:
        print(f"Atualizando carta - {name}")
        pool_collection.update_one({"_id": existing_card["_id"]}, {
                                   "$set": {"count": card["count"]}})
        return

    print(f"Adicionando carta - {name}")
    pool_collection.insert_one(
        {"name": name, "count": card["count"], "color": color, "scryfall_name": scryfall_name})

In [7]:
aggregate_query = [
    {"$project": {"allCards": {"$concatArrays": ["$maindeck", "$sideboard"]}}},
    {"$unwind": "$allCards"},
    {"$group": {"_id": {"deckId": "$_id", "name": "$allCards"}, "count": {"$sum": 1}}},
    {"$group": {"_id": "$_id.name", "count": {"$max": "$count"}}},
    {"$sort": {"_id": 1}},
    {"$project": {"name": "$_id", "count": "$count", "_id": 0}}
]

In [8]:
deck_collection = db_service.deck_collection()

cards = list(deck_collection.aggregate(aggregate_query))
for card in cards:
    if card["name"] == "Unknown Card":
        continue

    add_card_or_update(card)

Adicionando carta - Abrade
Adicionando carta - Abraded Bluffs
Adicionando carta - Abundant Growth
Adicionando carta - Accumulated Knowledge
Adicionando carta - Acorn Harvest
Adicionando carta - Active Volcano
Adicionando carta - Aerial Volley
Adicionando carta - Aethertow
Adicionando carta - Agony Warp
Adicionando carta - Akroan Crusader
Adicionando carta - Akroan Skyguard
Adicionando carta - All That Glitters
Adicionando carta - Alms of the Vein
Adicionando carta - Ancestral Anger
Adicionando carta - Ancestral Blade
Adicionando carta - Ancestral Mask
Adicionando carta - Ancient Den
Adicionando carta - Ancient Grudge
Adicionando carta - Ancient Stirrings
Adicionando carta - Angelsong
Adicionando carta - Annoyed Altisaur
Adicionando carta - Annul
Adicionando carta - Anoint with Affliction
Adicionando carta - Another Chance
Adicionando carta - Apostle's Blessing
Adicionando carta - Aqueous Form
Adicionando carta - Arashin Cleric
Adicionando carta - Arbor Elf
Adicionando carta - Arboreal 